In [64]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm, trange
import time
import torch 
from torch.optim import Adam 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences 
from sklearn.model_selection import train_test_split
from transformers import BertForTokenClassification
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [10]:
data = pd.read_csv('partial_labeled_sentences.csv')
data['compname'] = data['compname'].fillna(0)
ddf = data.groupby('sentence').mean().reset_index()[['sentence','compname']]

In [13]:
ddf = ddf.rename(columns = {'compname':'meanval'})
data = data.merge(ddf, on='sentence',how='inner')
data

,text,word,sentence,indexloc,compname,meanval
0,オリックス 銀行 と の 業務 提携 について,オリックス,1,0,1.0,0.285714
1,NaN,銀行,1,1,1.0,0.285714
2,NaN,と,1,2,0.0,0.285714
3,NaN,の,1,3,0.0,0.285714
4,NaN,業務,1,4,0.0,0.285714
...,...,...,...,...,...,...
24282,NaN,する,266,112,0.0,0.008547
24283,NaN,予定,266,113,0.0,0.008547
24284,NaN,です,266,114,0.0,0.008547
24285,NaN,。,266,115,0.0,0.008547


In [14]:
data = data[data['meanval'] > 0]

In [19]:
# data = data.rename(columns={'sentence':'Sentence #', 'compname':'Tag'})
data = data.rename(columns={'word':'Word'})

In [ ]:
d

In [79]:
data['Tag'] = data['Tag'].replace(1,'N').replace(0,'O')

In [131]:
print(data['text'].loc[0])
print()
print(data['text'].loc[7])
data.head(20)[['Word','Sentence #','Tag']]

オリックス 銀行 と の 業務 提携 について

20190510 当社 は オリックス 銀行 株式会社 と 、 オリックス 銀行 における 住宅 事業 者 向け サービス の 拡充 を 目的 に 顧客 紹介 業務 に関する 契約 を 締結 いたし まし た 。 本 業務 提携 を通じて 、 当社 は オリックス 銀行 より 紹介 を 受け た 住宅 事業 者 に対し 、 住宅 設備 の 延長 保証 を はじめ 定期 点検 や コール センター の 代行 など 「 人手 不足 」 に も 対応 できる 各種 サービス を 提供 し 、 オリックス 銀行 は これら を 通じ た 住宅 事業 者 向け サービス の 拡充 により 、 金融 機関 として 多様 化 する ニーズ に 対応 する 形 で 、 双方 協力 の もと 顧客 の 事業 活動 を 支援 いたし ます 。 当社 は 今後 も 住宅 事業 者 へ の ソリューション 提供 へ 真摯 に 向き合う 金融 機関 と の 提携 関係 を 強化 し 、 住宅 事業 者 の 幅広い ニーズ に お 応え する とともに 、 更 なる サービス 提供 体制 構築 を 進め て まい


,Word,Sentence #,Tag
0,オリックス,1,N
1,銀行,1,N
2,と,1,O
3,の,1,O
4,業務,1,O
5,提携,1,O
6,について,1,O
7,20190510,2,O
8,当社,2,O
9,は,2,O


In [132]:
len(data['Sentence #'].unique())

109

In [81]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data 
        self.empty = False 
        agg_func = lambda s: [(w,t) for w,t in zip(s['Word'].values.tolist(),
#                                                       s['POS'].values.tolist(),
                                                      s['Tag'].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
        
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1 
            return s
        except:
            return None

In [82]:
getter = SentenceGetter(data)

In [83]:
sentences = [' '.join([y[0] for y in x]) for x in getter.sentences]

In [84]:
sentences[0]

'オリックス 銀行 と の 業務 提携 について'

In [85]:
labels = [[y[1] for y in x] for x in getter.sentences]

In [86]:
labels[0]

['N', 'N', 'O', 'O', 'O', 'O', 'O']

In [87]:
tags_vals = list(set(data['Tag'].values))
tag2idx = {t:i for i,t in enumerate(tags_vals)}
idx2tag = {i: t for t, i in tag2idx.items()}
tag2idx

{'O': 0, 'N': 1}

In [88]:
MAX_LEN = 200
bs = 32

In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()

In [90]:
device

device(type='cpu')

In [35]:
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese')

In [91]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [92]:
tokenized_texts[0]

['オリックス', '銀行', 'と', 'の', '業務', '提携', 'について']

In [93]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                         maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

In [94]:
input_ids[0]

array([12036,  2269,    13,     5,  2363,  4802,   362,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [96]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                    maxlen=MAX_LEN, value=tag2idx['O'], padding='post',
                    dtype='long', truncating='post')

In [97]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [98]:
len(tr_masks)

76

In [99]:
# input_ids_u, input_ids_v, tags_u, tags_v = train_test_split(input_ids,
#                                                            tags,
#                                                            random_state=2018,
#                                                            test_size=0.015)

# attention_masks_u, attention_masks_v, _, _ = train_test_split(attention_masks, input_ids,
#                                             random_state=2018, test_size=0.015)


tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,
                                                           tags,
                                                           random_state=2018,
                                                           test_size=0.3)

tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                            random_state=2018, test_size=0.3)


In [145]:
import random
data_train, data_test, _, _ = train_test_split(sentences, list(range(len(sentences))), random_state=2018, test_size=0.3)
random.sample(data_test, 5)
                

['株式会社 アイセルネットワークス （ グループ ） と の 業務 提携 に関する お知らせ 令 和 元年 5 月 27 日 ダウンロード',
 'を 運営 する ウェルカム 社 と オイシックス・ラ・ 大地 が 資本 業務 提携 …',
 '2019 年 6 月 17 日 ホールディングス 株式会社 株式会社 ソフトフロントジャパンホールディングス と ソフトフロントジャパン 、 自然 会話 プラットフォーム 「 」 に関する 業務 提携 で 基本 合意 ～ コール センター 化 による コスト 削減 と 生産 性 向上 ソリューション で 協業 ～ ホールディングス 株式会社 （ 東京 都 港 区 、 代表 取締役 社長 ： 永易 俊彦 、 以下 、 ホールディングス ） と 、 株式会社 ソフト フロント ホールディングス （ 東京 都 千代田 区 、 代表 取締役 社長 ： 野田 亨 、 以下 、 ソフト フロント ホールディングス ） の 子会社 で ある 株式会社 ソフトフロントジャパン （ 東京 都 千代田 区 、 代表 取締役 社長 ： 髙須 英司 、 以下 、 ソフトフロントジャパン ） は 、 自然 会話 プラットフォーム 「 」 に関する 業務 提携 に関して 基本 合意 し まし た ので お知らせ し ます 。 ホールディングス は グループ 会社 （ 以下 、 グループ ） 体制 により 、 金融',
 '各位 2019 年 6 月 6 日 株式会社 デュアル タップ 代表 取締役 社長 臼井 貴弘 コード 3469 ／ 東証 二 部 「 」 を 運営 する と 業務 提携 スマホ ひとつ で “ 旅 する よう に 暮らす ” 「 」 に 物件 提供 東京 23 区 の 駅 近 に 特 化 し て 不動産 開発 を 行う 株式会社 デュアル タップ （ 東京 都 品川 区 ／ 代表 者 臼井 貴弘 ） は 、 急速 な 勢い で 取り扱い 物件 数 を 増やす インド の ホスピタリティ 会社 （ オヨ ） と ヤフー 株式会社 の 合弁 会社 として 、 「 （ オヨライフ ） 」 を 運営 する 株式会社 （ 東京 都 千代田 区 ／ 勝瀬 博則 ） と 物件 提供 に 向け た 業務 提携 を 締結 いたし ます 。 デュアル 

In [100]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [141]:
' '.join([indtoword[x][0] for x in np.array(val_inputs[0])])
# indtoword = list(tokenizer.vocab.items())
# indtoword[2000][0]

'各 ##位 2019 年 6 月 7 日 会社 名 ジェイ ・ エス ##コム ホールディングス 株式会社 代表 者 名 代表 取締役 社長 嶺 ##井 武 ##則 ( JAS ##DA ##Q ・ コード 3 7 7 9 ) 問 ##合 ##せ 先 業務 管理 統括 本部 課長 丸山 博 ##之 ( 電話 3 - 51 ##1 ##4 - 76 ##1 ) 業務 提携 に関する お ##知 ##らせ 当社 は 、 本 ##日 開催 の 取締役 会 において 、 以下 の とおり 中国 法人 江 [UNK] [UNK] [UNK] 科 ##技 有限 公司 ( 以下 、 「 掌 [UNK] 」 と いう 。 ) と の 間 で 、 中国 市場 において 、 当社 グループ が 取扱 ##う 良質 な 日本 製 化粧 品 を 掌 [UNK] が 有する インフル ##エン ##サー ##マー ##ケティング の 手法 を 活用 し 、 両社 の 更 なる 発展 と 収益 の 拡大 を 目的 と し た 業務 提携 を 行う こと について 決議 いた ##し まし た ので 、 お ##知 ##らせ いた ##し ます 。 1 . 業務 提携 の 理由 記 当社 グループ で は 日本 国内 において 、 理 美容 室 を 中心 と し た 化粧 品 の プロ [PAD] [PAD]'

In [101]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [102]:
model = BertForTokenClassification.from_pretrained('bert-base-japanese')

In [103]:
FULL_FINETUNING = False
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias','gamma','beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate':0.01},
        {'params':[p for n,p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate':0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{'params':[p for n,p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [104]:
from seqeval.metrics import f1_score

def flat_accuracy(valid_tags, pred_tags):

    """
    Define a flat accuracy metric to use while training the model.
    """

    return (np.array(valid_tags) == np.array(pred_tags)).mean()

def annot_confusion_matrix(valid_tags, pred_tags):

    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    # Create header from unique tags
    header = sorted(list(set(valid_tags + pred_tags)))

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


In [108]:
epoch

5

In [112]:
epochs = 5
max_grad_norm = 1.0
pad_tok = tokenizer.vocab["[PAD]"]
sep_tok = tokenizer.vocab["[SEP]"]
cls_tok = tokenizer.vocab["[CLS]"]
o_lab = tag2idx['O']
verbose = True
# epoch = 0

for _ in trange(epochs, desc='Epoch'):
    epoch += 1
    print('starting train loop')
    model.train()
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], [] 
    
    for step, batch in enumerate(train_dataloader):
        #add batch to gpu (cpu probably)
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        #forward pass 
        outputs = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels,
        )
        loss, tr_logits = outputs[:2]
        print('backward pass')
        #backward pass
        loss.backward()
        #compute train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        #subset out unwanted predictions on CLS/PAD/SEP tokens
        preds_mask = (
            (b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok)
        )
        
        tr_logits = tr_logits.detach().cpu().numpy()
        tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
        tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1)
        tr_batch_labels = tr_label_ids.to('cpu').numpy()
        tr_preds.extend(tr_batch_preds)
        tr_labels.extend(tr_batch_labels)
        
        #compute training accuracy
        tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)
        tr_accuracy += tmp_tr_accuracy
        
        #gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        #update parameters
        optimizer.step()
        model.zero_grad()
        
    tr_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    
    #print training loss and accuracy per epoch 
    print(f"Train loss: {tr_loss}")
    print(f"Train accuracy: {tr_accuracy}")
    
    #validation loop 
    print('starting validation loop')
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions, true_labels = [], []
    
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            tmp_eval_loss, logits = outputs[:2]
        
        #subset out unwanted predictions on cls/pad/sep tokens
        preds_mask = (
            (b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok)
        )
        
        logits = logits.detach().cpu().numpy()
        label_ids = torch.masked_select(b_labels, (preds_mask == 1))
        val_batch_preds = np.argmax(logits[preds_mask.squeeze(1)], axis=1)
        val_batch_labels = label_ids.to('cpu').numpy()
        predictions.extend(val_batch_preds)
        true_labels.extend(val_batch_labels)
        
        tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy 
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
        
    #evaluate loss, acc, conf matrix and class report on devset 
    pred_tags = [idx2tag[i] for i in predictions]
    valid_tags = [idx2tag[i] for i in true_labels]
    cl_report = classification_report(valid_tags, pred_tags)
    conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
    eval_loss = eval_loss / nb_eval_steps 
    eval_accuracy = eval_accuracy / nb_eval_steps 
    
    #report metrics 
    print(f"Validation loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    print(f"Classificatiotn Report:\m {cl_report}")
    if verbose:
        print(f"Confusion Matrix:\n {conf_mat}")
        
    #save model and optimizer state_dict following every epoch 
    save_path = f"models/train_checkpoint_epoch_{epoch}.tar"
    torch.save(
        {
            "epoch":epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_loss": tr_loss,
            "train_acc": tr_accuracy,
            "eval_loss":eval_loss,
            "eval_acc": eval_accuracy,
            "classification_report": cl_report,
            "confusion_matrix": conf_mat,
        },
        save_path,
    )
    print(f"Checkpoint saved to {save_path}")




Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

starting train loop
backward pass
backward pass
backward pass
Train loss: 0.5859988530476888
Train accuracy: 0.7378427576364541
starting validation loop
Validation loss: 0.5473500192165375
Validation Accuracy: 0.8728249511607833
Classificatiotn Report:\m            precision    recall  f1-score   support

        N       0.02      0.11      0.03        91

micro avg       0.02      0.11      0.03        91
macro avg       0.02      0.11      0.03        91

Confusion Matrix:
 	N O
N	[27 80]
O	[ 757 3160]





Epoch:  20%|██        | 1/5 [01:25<05:43, 85.77s/it]

Checkpoint saved to models/train_checkpoint_epoch_11.tar
starting train loop
backward pass
backward pass
backward pass
Train loss: 0.579609215259552
Train accuracy: 0.7497346598374471
starting validation loop
Validation loss: 0.5390204787254333
Validation Accuracy: 0.8815705783471901
Classificatiotn Report:\m            precision    recall  f1-score   support

        N       0.02      0.10      0.03        91

micro avg       0.02      0.10      0.03        91
macro avg       0.02      0.10      0.03        91

Confusion Matrix:
 	N O
N	[25 82]
O	[ 685 3232]





Epoch:  40%|████      | 2/5 [02:51<04:16, 85.65s/it]

...

Checkpoint saved to models/train_checkpoint_epoch_13.tar
starting train loop
backward pass
backward pass
backward pass
Train loss: 0.5643460154533386
Train accuracy: 0.7651613528935526
starting validation loop
Validation loss: 0.5227641761302948
Validation Accuracy: 0.8968129571577848
Classificatiotn Report:\m            precision    recall  f1-score   support

        N       0.02      0.09      0.03        91

micro avg       0.02      0.09      0.03        91
macro avg       0.02      0.09      0.03        91

Confusion Matrix:
 	N O
N	[19 88]
O	[ 557 3360]





Epoch:  80%|████████  | 4/5 [05:42<01:25, 85.71s/it]

Checkpoint saved to models/train_checkpoint_epoch_14.tar
starting train loop
backward pass
backward pass
backward pass
Train loss: 0.5575761993726095
Train accuracy: 0.7765062524696917
starting validation loop
Validation loss: 0.5148117244243622
Validation Accuracy: 0.9036845213756759
Classificatiotn Report:\m            precision    recall  f1-score   support

        N       0.03      0.11      0.04        91

micro avg       0.03      0.11      0.04        91
macro avg       0.03      0.11      0.04        91

Confusion Matrix:
 	N O
N	[18 89]
O	[ 501 3416]





Epoch: 100%|██████████| 5/5 [07:08<00:00, 85.72s/it]

Checkpoint saved to models/train_checkpoint_epoch_15.tar


In [114]:
model_name = 'bert-base-japanese'
checkpoint = torch.load('models/train_checkpoint_epoch_10.tar')
model_state_dict = checkpoint["model_state_dict"]
model = BertForTokenClassification.from_pretrained(
    model_name, num_labels=model_state_dict['classifier.weight'].shape[0]
)
model.load_state_dict(model_state_dict)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [153]:

    input_text = random.choice(data_test)
    encoded_text = tokenizer.encode(input_text)
    wordpieces = [tokenizer.decode(tok).replace(' ','') for tok in encoded_text]

    input_ids = torch.tensor(encoded_text).unsqueeze(0).long()
    labels = torch.tensor([1] * input_ids.size(1)).unsqueeze(0).long()
    outputs = model(input_ids, labels=labels)
    loss, scores = outputs[:2]
    scores = scores.detach().numpy()

    label_ids = np.argmax(scores, axis=2)
    preds = [idx2tag[i] for i in label_ids[0]]

    wp_preds = list(zip(wordpieces, preds))
    toplevel_preds = [pair[1] for pair in wp_preds if '##' not in pair[0]]
    str_rep = ' '.join([t[0] for t in wp_preds]).replace(' ##', '').split()

    if len(str_rep) == len(toplevel_preds):
        preds_final = list(zip(str_rep, toplevel_preds))
        b_preds_df = pd.DataFrame(preds_final)
        b_preds_df.columns = ['text','pred']
        for tag in ['O','N']:
            b_preds_df[f"b_pred_{tag}"] = np.where(
                b_preds_df['pred'].str.contains(tag), 1, 0
            )
    else:
        print('could not match up output string with preds.')
    print(input_text)
    b_preds_df.loc[:,'text':'pred']


株式会社 オウケイウェイヴ について ※ 記載 さ れ た 商品 名 、 製品 名 は 各社 の 登録 商標 または 商標 です 。 © 2019 「 感謝 経済 」 プラットフォーム の グローバル 戦略 を 推進 この たび の 社 と の 資本 ・ 業務 提携 は 、 世界 的 に 決済 の キャッシュ レス 化 が 進展 する 中 で 、 暗号 資産 （ 仮想 通貨 ） を 決済 に 利用 できる デビット カード の 発行 ならびに 仮想 通貨 取引 所 の 運営 を 主力 事業 として サービスローンチ に 向け て 準備 を 進める 新興 企業 社 と の 協業 により 、 当社 が 推進 する 今後 、 当社 は グループ 会社 を通じて 社 と 下記 の 協業 を 推進 し ます 。 当社 は 社 の マーケティング パートナー として 、 当社 米国 子会社 （ 本社 ： アメリカ合衆国 カリフォルニア 州 サンフランシスコ ） が 北米 を 中心 と する 英語 圏 にて 展開 する デジタルグリーティングカードサービス 「 ブロック チェーン 運用 コンサルティ


,text,pred
0,[CLS],O
1,株式会社,O
2,オウケイウェイヴ,N
3,について,O
4,※,N
...,...,...
155,ブロック,N
156,チェーン,O
157,運用,N
158,コンサルティ,O


In [118]:
print(checkpoint['confusion_matrix'])

	N O
N	[28 79]
O	[ 811 3106]


In [136]:
for i in range(1,11):
    checkpoint = torch.load('models/train_checkpoint_epoch_{}.tar'.format(str(i)))
    print(i)
    print(checkpoint['eval_acc'])
    print(checkpoint['confusion_matrix'])
    print()

1
0.7457748398528008
	N O
N	[48 59]
O	[1614 2303]

2
0.7572690927263641
	N O
N	[47 60]
O	[1521 2396]

3
0.7917404933896688
	N O
N	[45 62]
O	[1424 2493]

4
0.8042342465131072
	N O
N	[43 64]
O	[1322 2595]

5
0.8144791240743265
	N O
N	[39 68]
O	[1236 2681]

6
0.8268479396665305
	N O
N	[38 69]
O	[1136 2781]

7
0.8373426922902185
	N O
N	[37 70]
O	[1051 2866]

8
0.848587070101313
	N O
N	[30 77]
O	[ 954 2963]

9
0.8564581345690792
	N O
N	[29 78]
O	[ 890 3027]

10
0.866203262005361
	N O
N	[28 79]
O	[ 811 3106]

